In [3]:
import pickle
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import os
import geopandas as gpd
import rasterio
from rasterio.features import shapes

# Original grid

In [4]:
grid_x, grid_y = np.meshgrid(np.arange(1121),np.arange(881))
grid_xy = grid_x*10**4 + (880-grid_y) + 10**8
mask = None
with rasterio.Env():
    with rasterio.open('/storage/home/hcoda1/6/njadidoleslam3/p-rbras6-0/projects/stochsm/data/nws_precip_1day_20210921_conus.tif') as src:
        image = src.read(1) # first band
        image = grid_xy.astype(np.int32)
        results = (
        {'properties': {'raster_val': grid_xy.ravel()[i]}, 'geometry': s}
        for i, (s, v) in enumerate(
            shapes(image,  transform=src.transform)))
geoms = list(results)
gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
gpd_polygonized_raster.columns = ['geometry', 'grid_xy']
gpd_polygonized_raster['grid_xy'] = gpd_polygonized_raster['grid_xy'].astype(np.int64)
gpd_polygonized_raster.crs = "+proj=stere +lat_0=90 +lat_ts=60 +lon_0=-105 +x_0=0 +y_0=0 +a=6371200 +b=6371200 +units=m +no_defs"
gpd_polygonized_raster = gpd_polygonized_raster.to_crs(4326)


# Masked grid

In [4]:
fn_st_grid_masked = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/data/gis_files/st4/stage4_grid_new.shp'
st4_grid_masked = gpd.read_file(fn_st_grid_masked)


In [5]:
data_test = []
with open('/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/events/2019.pickle', 'rb') as handle:
    try:
        while True:
            data_test.append(pickle.load(handle))
    except EOFError:
        pass

In [1]:
import pickle
import numpy as np
import pandas as pd
import os
# Reorganize data by month
def read_pickle(fn):
    data_test = []
    with open(fn, 'rb') as handle:
        try:
            while True:
                data_test.append(pickle.load(handle))
        except EOFError:
            pass
    return data_test


#####
# SHAPE FILES  TO BE ADDED TO GPM ALGORITHM
# fn_st_grid_masked = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/data/gis_files/st4/stage4_grid_new.shp'
# st4_grid_masked = gpd.read_file(fn_st_grid_masked)
#####
pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/monthly/{month}.pickle'
in_pick_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/events/{year}.pickle'
#
#####
# month = int(sys.argv[1])
month = 1

### Generate year list and shuffle for avoiding I/O problems
year_list = np.arange(2000,2021)
np.random.seed()
np.random.shuffle(year_list)
###

fn_out_pickle = pickle_fmt.format(month = month)

# with open(fn_out_pickle, 'wb') as handle:
events_all = []
for year in year_list:
    in_pickle = in_pick_fmt.format(year = year)
    data_test = read_pickle(in_pickle)

    # _events_all = []
    for i_y in range(len(data_test)): # loop over grid_y
        for i_x in range(len(data_test[i_y])): # loop over grid_x
            grid_xy = int(data_test[i_y][i_x][0])
            idx = data_test[i_y][i_x][3][0] == month
            n_events = sum(idx)
            if n_events >0:
                event_dur = data_test[i_y][i_x][3][1][idx]
                dry_vec = data_test[i_y][i_x][3][2][idx]
                intensity = data_test[i_y][i_x][3][3][idx]
                for i in range(n_events):
                    events_all.append([grid_xy, event_dur[i],intensity[i],dry_vec[i]])
with open(fn_out_pickle, 'wb') as handle:
    pickle.dump(events_all, handle, protocol= pickle.HIGHEST_PROTOCOL) 



events_all = pd.DataFrame(events_all, columns=['grid_xy','tr', 'i', 'tb'])
events_all['h'] = events_all['i'] * events_all['tr']
summary = events_all.groupby('grid_xy').mean().reset_index()
counts = events_all.groupby('grid_xy').count().reset_index()
var_h = events_all.groupby('grid_xy').var().reset_index() # standard deviation of storm depths
mean_h = events_all.groupby('grid_xy').mean().reset_index() # standard deviation of storm depths
summary['lambda'] = mean_h['h']/var_h['h']
summary['kappa'] = mean_h['h']* summary['lambda']
summary['count'] = counts['tr']/21

#### SAVE summary data to pickle files 
summary_path = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary'
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

fn_summary = os.path.join(summary_path, '{month}.pickle'.format(month = month))
with open(fn_summary, 'wb') as handle:
    pickle.dump(summary, handle, protocol= pickle.HIGHEST_PROTOCOL) 


## END

In [227]:
year_list[0:1]

array([2006])

In [223]:
events_all = pd.DataFrame(events_all, columns=['grid_xy','tr', 'i', 'tb'])
events_all['h'] = events_all['i'] * events_all['tr']
summary = events_all.groupby('grid_xy').mean().reset_index()
counts = events_all.groupby('grid_xy').count().reset_index()
var_h = events_all.groupby('grid_xy').var().reset_index() # standard deviation of storm depths
mean_h = events_all.groupby('grid_xy').mean().reset_index() # standard deviation of storm depths
summary['lambda'] = mean_h['h']/var_h['h']
summary['kappa'] = mean_h['h']* summary['lambda']
summary['count'] = counts['tr']/21

#### SAVE summary data to pickle files 
summary_path = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary'
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

fn_summary = os.path.join(summary_path, '{month}.pickle'.format(month = month))
with open(fn_summary, 'wb') as handle:
    pickle.dump(summary, handle, protocol= pickle.HIGHEST_PROTOCOL) 


In [198]:
events_all = pd.DataFrame(events_all, columns=['grid_xy','tr', 'i', 'tb'])
events_all['h'] = events_all['i'] * events_all['tr']
summary = events_all.groupby('grid_xy').mean().reset_index()
counts = events_all.groupby('grid_xy').count().reset_index()
var_h = events_all.groupby('grid_xy').var().reset_index() # standard deviation of storm depths
mean_h = events_all.groupby('grid_xy').mean().reset_index() # standard deviation of storm depths
summary['lambda'] = mean_h['h']/var_h['h']
summary['kappa'] = mean_h['h']* summary['lambda']
summary['count'] = counts['tr']



In [5]:
summary

,grid_xy,tr,i,tb,h,lambda,kappa,count
0,100160467,84.0,0.320417,36.0,28.740000,0.036212,1.040725,0.190476
1,100160468,91.0,1.182288,62.0,39.763333,0.019160,0.761862,0.142857
2,100160469,6.0,2.991667,99.0,17.950000,NaN,NaN,0.047619
3,100160470,9.0,2.004444,93.0,18.040000,NaN,NaN,0.047619
4,100160512,9.0,2.445556,91.0,22.010000,NaN,NaN,0.047619
...,...,...,...,...,...,...,...,...
411642,110630167,19.8,0.786720,93.2,2.736000,0.406180,1.111308,0.238095
411643,110710140,2.0,0.370000,7.0,0.740000,NaN,NaN,0.047619
411644,110720143,2.0,4.995000,7.0,9.990000,NaN,NaN,0.047619
411645,110730141,1.0,8.750000,8.0,8.750000,NaN,NaN,0.047619


In [7]:
# mit_array = pd.DataFrame({'grid_xy':[int(x) for x in summary[:,0]], 'mean_it':summary[:,1]})

summary['grid_xy'] = summary['grid_xy'].astype(np.int64)
data = st4_grid_masked.merge(summary)
# data['mean_it'] = data['mean_it'].astype(np.float16)

In [8]:
data

,grid_xy,geometry,tr,i,tb,h,lambda,kappa,count
0,109830819,"POLYGON ((-68.20350 47.32350, -68.25058 47.347...",48.000000,0.094722,30.166667,8.280000,0.038490,0.318700,0.285714
1,109840819,"POLYGON ((-68.15648 47.29963, -68.20350 47.323...",52.800000,0.109333,41.000000,9.840000,0.038402,0.377880,0.238095
2,109850819,"POLYGON ((-68.10951 47.27574, -68.15648 47.299...",48.000000,0.092500,30.166667,8.400000,0.035674,0.299658,0.285714
3,109860819,"POLYGON ((-68.06261 47.25183, -68.10951 47.275...",60.000000,0.135000,57.250000,13.080000,0.034304,0.448701,0.190476
4,109870819,"POLYGON ((-68.01576 47.22789, -68.06261 47.251...",52.800000,0.111333,41.000000,10.704000,0.032036,0.342911,0.238095
...,...,...,...,...,...,...,...,...,...
406819,106050049,"POLYGON ((-97.44511 25.85003, -97.44827 25.859...",21.750000,0.166169,108.500000,2.427500,0.424164,1.029658,0.190476
406820,106060049,"POLYGON ((-97.43133 25.84411, -97.42655 25.876...",16.000000,0.330714,114.250000,2.682500,0.418068,1.121467,0.190476
406821,106070049,"POLYGON ((-97.39508 25.83941, -97.39023 25.872...",19.428571,0.350857,71.571429,2.141429,0.241854,0.517913,0.333333
406822,106060048,"POLYGON ((-97.42264 25.84038, -97.43017 25.843...",17.000000,0.218679,102.200000,2.342000,0.302816,0.709195,0.238095


In [9]:

fn_temp = 'mean_it_{year}_test.png'
cm1 = plt.cm.get_cmap('Reds',10)
fig, ax = plt.subplots(figsize=(10, 8))
# year = 2019
# st4_grid.plot(ax=ax, facecolor="none", edgecolor='black', zorder= 5, alpha= 1 )
a = data.plot(ax=ax, column='tb', edgecolor='none', cmap=cm1, vmin = 0, vmax=200)
ax.set_axis_off()
# ax.invert_yaxis()
plt.title(str(year), fontsize=25)
cax = fig.add_axes([0.1, 0.1, 0.8, 0.04])
fig.colorbar( a.collections[0], cax=cax, orientation='horizontal')
cax.set_xlabel('Mean Interarrival Time', fontsize=18)
cax.tick_params(labelsize=16)

cax.tick_params(labelsize=16)
cax.locator_params(nbins=11)
fn_out = os.path.join('/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/trash',  fn_temp.format(year = year))
fig.savefig(fn_out, dpi=300, bbox_inches='tight')
plt.close(fig)

In [ ]:
for variable in var_list:
    out_pth = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/monthly/{variable}/'.format(variable = variable)
    out_fn = '{variable}_{month}.jpg'.format(variable = variable, month=month)
    if not os.path.exists(out_pth):
        os.makedirs(out_pth)
    
    fn_temp = '{variable}.png'
    cm1 = plt.cm.get_cmap('Reds',10)
    fig, ax = plt.subplots(figsize=(10, 8))
    year = 2019
    # st4_grid.plot(ax=ax, facecolor="none", edgecolor='black', zorder= 5, alpha= 1 )
    a = data.plot(ax=ax, column='tb', edgecolor='none', cmap=cm1, vmin = 0, vmax=200)
    ax.set_axis_off()
    # ax.invert_yaxis()
    plt.title(str(year), fontsize=25)
    cax = fig.add_axes([0.1, 0.1, 0.8, 0.04])
    fig.colorbar( a.collections[0], cax=cax, orientation='horizontal')
    cax.set_xlabel('Mean Interarrival Time', fontsize=18)
    cax.tick_params(labelsize=16)

    cax.tick_params(labelsize=16)
    cax.locator_params(nbins=11)
    fn_out = os.path.join(out_pth,out_fn)
    fig.savefig(fn_out, dpi=300, bbox_inches='tight')
    plt.close(fig)